In [1]:
import os
import librosa
# import pydub
from pydub import AudioSegment
import numpy as np
import copy
import random
from scipy import signal as scipy_signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import datetime

In [2]:
dataset_dir = '/home/torooc/dataHDD2/speech_commands_v0.01/'

In [3]:
testing_list_path = dataset_dir + 'testing_list.txt'
validation_list_path = dataset_dir + 'validation_list.txt'

In [4]:
with open(testing_list_path) as f:
    testing_list = f.readlines()
    testing_list = [file.strip() for file in testing_list]
    
with open(validation_list_path) as f:
    validation_list = f.readlines()
    validation_list = [file.strip() for file in validation_list]

In [5]:
command_dirs = os.listdir(dataset_dir)

command_dirs = [direc for direc in command_dirs if os.path.isdir(os.path.join(dataset_dir, direc))]

command_dirs.sort()

command_dirs.remove('_background_noise_')

In [6]:
print(command_dirs)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [7]:
key2label_dict = dict()
label2key_list = list()

key2label_dict[' '] = 0
label2key_list.append(' ')

for i, command in enumerate(command_dirs):
    key2label_dict[command] = i + 1
    label2key_list.append(command)

In [8]:
key2label_dict

{' ': 0,
 'bed': 1,
 'bird': 2,
 'cat': 3,
 'dog': 4,
 'down': 5,
 'eight': 6,
 'five': 7,
 'four': 8,
 'go': 9,
 'happy': 10,
 'house': 11,
 'left': 12,
 'marvin': 13,
 'nine': 14,
 'no': 15,
 'off': 16,
 'on': 17,
 'one': 18,
 'right': 19,
 'seven': 20,
 'sheila': 21,
 'six': 22,
 'stop': 23,
 'three': 24,
 'tree': 25,
 'two': 26,
 'up': 27,
 'wow': 28,
 'yes': 29,
 'zero': 30}

In [9]:
total_wav_file_list = list()

for command_dir in command_dirs:
    wav_list = os.listdir(os.path.join(dataset_dir, command_dir))
    
    wav_list_with_prefix = [os.path.join(command_dir, file_name) for file_name in wav_list]
    
    total_wav_file_list += wav_list_with_prefix

In [10]:
training_list = list(set(total_wav_file_list) - set(testing_list) - set(validation_list))
training_list.sort()


In [11]:
'''
validation_list
testing_list
training_list
'''

'\nvalidation_list\ntesting_list\ntraining_list\n'

In [12]:
audio_metadata = list()

for wav_file_name in total_wav_file_list:
    audio = AudioSegment.from_wav(dataset_dir + wav_file_name)
    fs = audio.frame_rate
    length_in_samples = audio.frame_count()
    
#   normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
#   print("Max: {}, Min: {}".format(max(normalized_audio_array), min(normalized_audio_array)))
    
    audio_metadata.append([fs, length_in_samples])

In [13]:
SR = 16000

class DatasetLoader():
    
    def __init__(self, batch_size, dataset_dir, wav_file_name_list, key2label, label2key):
        self.batch_size = batch_size
        self.dataset_dir = dataset_dir
        self.wav_file_name_list = copy.deepcopy(wav_file_name_list)
        self.sr = SR
        self.nsc_in_ms = 40
#         self.nov_in_ms = self.nsc_in_ms / 2
        self.nov_in_ms = 0
        self.nsc_in_sample = int(self.nsc_in_ms / 1000 * self.sr)
        self.nov_in_sample = int(self.nov_in_ms / 1000 * self.sr)
        
        self.key2label = key2label
        self.label2key = label2key
        
    def shuffle_dataset_order(self):
        random.shuffle(self.wav_file_name_list)
        
    def batch_generator(self):
        
        self.shuffle_dataset_order()
        
        spectrogram_list = list()
        label_list = list()
        
        for i, wav_file_name in enumerate(self.wav_file_name_list):
            
            audio = AudioSegment.from_wav(self.dataset_dir + wav_file_name)
            
            normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
            f, t, Zxx = scipy_signal.stft(normalized_audio_array, fs=self.sr, 
                                          nperseg=self.nsc_in_sample,
                                          noverlap=self.nov_in_sample)
        
            Sxx = np.abs(Zxx)
            
            normalized_spectrogram = (20 * np.log10(np.maximum(Sxx, 1e-8)) + 160) / 160
            spectrogram_list.append(normalized_spectrogram)
            
            keyword = wav_file_name.split('/')[0]
            label = self.key2label[keyword]
            label_list.append(label)
            

            if (i + 1) % self.batch_size == 0 or i + 1 == len(self.wav_file_name_list):
                
                spectrogram_time_step_list = [specgram.shape[1] for specgram in spectrogram_list]
                
                max_time_step = max(spectrogram_time_step_list)
                
                freq_size = spectrogram_list[0].shape[0]
                
                batch = np.zeros([len(spectrogram_list), freq_size, max_time_step])
                
                for j, specgram in enumerate(spectrogram_list):
                    batch[j, :specgram.shape[0], :specgram.shape[1]] = specgram
                
                spectrogram_list = list()
                
#                 print('{}:{}'.format(i, batch.shape[0]))
            
                yield batch, np.asarray(label_list)
                
                label_list = list()
                
#                 break
    
#             break

In [14]:
len(training_list)

51088

In [15]:
# dataset_loader_training = DatasetLoader(256, dataset_dir, training_list, key2label_dict, label2key_list)
# dataset_loader_testing = DatasetLoader(256, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(256, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_testing = DatasetLoader(64, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(64, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_training = DatasetLoader(768, dataset_dir, training_list, key2label_dict, label2key_list)
# dataset_loader_testing = DatasetLoader(768, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(768, dataset_dir, validation_list, key2label_dict, label2key_list)

dataset_loader_training = DatasetLoader(256, dataset_dir, training_list, key2label_dict, label2key_list)
dataset_loader_testing = DatasetLoader(256, dataset_dir, testing_list, key2label_dict, label2key_list)
dataset_loader_validation = DatasetLoader(256, dataset_dir, validation_list, key2label_dict, label2key_list)

In [16]:
# batch_generator = dataset_loader_training.batch_generator()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# class basic_model(nn.Module):
#     def __init__(self, D_in, H, num_layers, num_labels):
#         super(basic_model, self).__init__()
#         self.fc = torch.nn.Linear(D_in, H)
#         self.relu = torch.nn.ReLU()
#         self.dropout = nn.Dropout(p=0.2)

#         self.gru_layers = nn.ModuleList([nn.GRU(H, int(H / 2), bidirectional=True, batch_first=True) for i in range(num_layers)])

#         self.fc_pred = nn.Linear(H, num_labels)
#         self.log_softmax = nn.LogSoftmax(dim=2)
        
#     def forward(self, input_tensor):
#         # (B, T, F)
#         output_tensor = self.fc(input_tensor)
#         output_tensor = self.relu(output_tensor)
#         output_tensor = self.dropout(output_tensor)
#         # (B, T, H)
#         for layer in self.gru_layers:
#             output_tensor, _ = layer(output_tensor)
            
#         output_tensor = self.fc_pred(output_tensor)

#         output_tensor = self.log_softmax(output_tensor)
        
#         return output_tensor

class basic_conv_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_conv_model, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 16, (13, 2))
        
        self.conv2d_2 = nn.Conv2d(16, 32, (13, 2))
        
        self.flatten = nn.Flatten()
        
        self.fc = nn.Linear(4800, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)
        tensor = self.conv2d_1(input_tensor)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        tensor = self.conv2d_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor
        
    
# model = basic_model(321, 512, 3, 31).float().to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# dictation_loss = nn.CTCLoss().to(device)

model = basic_conv_model().float().to(device)  
optimizer = optim.Adam(model.parameters(), lr=1e-4)
        

In [19]:
MAX_EPOCH = 300

for epoch in range(MAX_EPOCH):
    training_loss_list = list()
    batch_generator = dataset_loader_training.batch_generator()
    model.train()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
        optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
        loss.backward()
        training_loss_list.append(loss.item())
        optimizer.step()
        
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_train = correct_num / total_num

    testing_loss_list = list()

    batch_generator = dataset_loader_testing.batch_generator()

    model.eval()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        testing_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_test = correct_num / total_num

    validation_loss_list = list()

    batch_generator = dataset_loader_validation.batch_generator()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        validation_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_valid = correct_num / total_num
    
    training_loss_mean = np.mean(training_loss_list)
    testing_loss_mean = np.mean(testing_loss_list)
    validation_loss_mean = np.mean(validation_loss_list)
    
    print(datetime.datetime.now().time())
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, training_loss_mean, testing_loss_mean, validation_loss_mean))
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, acc_train, acc_test, acc_valid))
    

07:31:09.487505
#000 [Train: 3.315] [Test: 3.136] [Valid: 3.117]
#000 [Train: 0.077] [Test: 0.115] [Valid: 0.112]
07:32:02.192068
#001 [Train: 2.950] [Test: 2.813] [Valid: 2.759]
#001 [Train: 0.171] [Test: 0.195] [Valid: 0.215]
07:32:53.938792
#002 [Train: 2.728] [Test: 2.674] [Valid: 2.601]
#002 [Train: 0.234] [Test: 0.244] [Valid: 0.255]
07:33:47.472152
#003 [Train: 2.577] [Test: 2.554] [Valid: 2.483]
#003 [Train: 0.283] [Test: 0.285] [Valid: 0.314]
07:34:39.837831
#004 [Train: 2.452] [Test: 2.413] [Valid: 2.350]
#004 [Train: 0.322] [Test: 0.332] [Valid: 0.358]
07:35:32.030735
#005 [Train: 2.337] [Test: 2.367] [Valid: 2.294]
#005 [Train: 0.352] [Test: 0.343] [Valid: 0.357]
07:36:24.984309
#006 [Train: 2.244] [Test: 2.271] [Valid: 2.215]
#006 [Train: 0.382] [Test: 0.371] [Valid: 0.392]
07:37:18.214838
#007 [Train: 2.161] [Test: 2.189] [Valid: 2.126]
#007 [Train: 0.404] [Test: 0.405] [Valid: 0.419]
07:38:10.690531
#008 [Train: 2.094] [Test: 2.135] [Valid: 2.088]
#008 [Train: 0.425] [Te

08:34:04.796766
#072 [Train: 0.619] [Test: 1.024] [Valid: 0.955]
#072 [Train: 0.830] [Test: 0.739] [Valid: 0.750]
08:34:56.649918
#073 [Train: 0.614] [Test: 1.017] [Valid: 0.956]
#073 [Train: 0.830] [Test: 0.734] [Valid: 0.753]
08:35:48.667319
#074 [Train: 0.606] [Test: 1.031] [Valid: 0.953]
#074 [Train: 0.833] [Test: 0.734] [Valid: 0.751]
08:36:41.074254
#075 [Train: 0.603] [Test: 1.018] [Valid: 0.942]
#075 [Train: 0.834] [Test: 0.735] [Valid: 0.753]
08:37:34.067201
#076 [Train: 0.595] [Test: 1.020] [Valid: 0.942]
#076 [Train: 0.836] [Test: 0.736] [Valid: 0.751]
08:38:26.431262
#077 [Train: 0.594] [Test: 1.020] [Valid: 0.949]
#077 [Train: 0.837] [Test: 0.732] [Valid: 0.752]
08:39:18.576501
#078 [Train: 0.585] [Test: 1.021] [Valid: 0.945]
#078 [Train: 0.839] [Test: 0.738] [Valid: 0.753]
08:40:10.800070
#079 [Train: 0.582] [Test: 1.008] [Valid: 0.938]
#079 [Train: 0.839] [Test: 0.740] [Valid: 0.759]
08:41:02.933240
#080 [Train: 0.576] [Test: 1.013] [Valid: 0.942]
#080 [Train: 0.842] [Te

09:36:45.384895
#144 [Train: 0.344] [Test: 1.092] [Valid: 0.972]
#144 [Train: 0.907] [Test: 0.747] [Valid: 0.759]
09:37:37.603691
#145 [Train: 0.340] [Test: 1.097] [Valid: 0.971]
#145 [Train: 0.908] [Test: 0.748] [Valid: 0.761]
09:38:30.327456
#146 [Train: 0.340] [Test: 1.092] [Valid: 0.966]
#146 [Train: 0.908] [Test: 0.747] [Valid: 0.762]
09:39:22.671680
#147 [Train: 0.335] [Test: 1.104] [Valid: 0.967]
#147 [Train: 0.910] [Test: 0.747] [Valid: 0.765]
09:40:15.233911
#148 [Train: 0.335] [Test: 1.116] [Valid: 0.990]
#148 [Train: 0.910] [Test: 0.742] [Valid: 0.755]
09:41:07.440490
#149 [Train: 0.334] [Test: 1.109] [Valid: 0.994]
#149 [Train: 0.909] [Test: 0.743] [Valid: 0.759]
09:42:00.814398
#150 [Train: 0.331] [Test: 1.101] [Valid: 0.989]
#150 [Train: 0.910] [Test: 0.747] [Valid: 0.763]
09:42:52.791716
#151 [Train: 0.327] [Test: 1.133] [Valid: 0.993]
#151 [Train: 0.911] [Test: 0.742] [Valid: 0.757]
09:43:44.140682
#152 [Train: 0.328] [Test: 1.107] [Valid: 0.978]
#152 [Train: 0.911] [Te

10:39:28.384359
#216 [Train: 0.202] [Test: 1.298] [Valid: 1.145]
#216 [Train: 0.948] [Test: 0.742] [Valid: 0.755]
10:40:20.444749
#217 [Train: 0.203] [Test: 1.309] [Valid: 1.141]
#217 [Train: 0.947] [Test: 0.744] [Valid: 0.761]
10:41:12.110966
#218 [Train: 0.201] [Test: 1.310] [Valid: 1.127]
#218 [Train: 0.948] [Test: 0.743] [Valid: 0.757]
10:42:04.174827
#219 [Train: 0.202] [Test: 1.320] [Valid: 1.144]
#219 [Train: 0.948] [Test: 0.741] [Valid: 0.758]
10:42:56.400183
#220 [Train: 0.201] [Test: 1.320] [Valid: 1.118]
#220 [Train: 0.947] [Test: 0.744] [Valid: 0.757]
10:43:48.898824
#221 [Train: 0.199] [Test: 1.329] [Valid: 1.128]
#221 [Train: 0.948] [Test: 0.742] [Valid: 0.762]
10:44:41.007749
#222 [Train: 0.195] [Test: 1.313] [Valid: 1.141]
#222 [Train: 0.949] [Test: 0.740] [Valid: 0.760]
10:45:33.458883
#223 [Train: 0.196] [Test: 1.355] [Valid: 1.136]
#223 [Train: 0.950] [Test: 0.737] [Valid: 0.758]
10:46:26.056182
#224 [Train: 0.193] [Test: 1.347] [Valid: 1.136]
#224 [Train: 0.949] [Te

11:40:33.509149
#288 [Train: 0.123] [Test: 1.605] [Valid: 1.350]
#288 [Train: 0.970] [Test: 0.735] [Valid: 0.751]
11:41:22.912464
#289 [Train: 0.125] [Test: 1.592] [Valid: 1.351]
#289 [Train: 0.969] [Test: 0.734] [Valid: 0.756]
11:42:12.313010
#290 [Train: 0.126] [Test: 1.624] [Valid: 1.372]
#290 [Train: 0.968] [Test: 0.737] [Valid: 0.751]
11:43:01.742109
#291 [Train: 0.121] [Test: 1.599] [Valid: 1.367]
#291 [Train: 0.970] [Test: 0.738] [Valid: 0.756]
11:43:51.147186
#292 [Train: 0.123] [Test: 1.621] [Valid: 1.364]
#292 [Train: 0.969] [Test: 0.735] [Valid: 0.754]
11:44:40.553863
#293 [Train: 0.118] [Test: 1.629] [Valid: 1.374]
#293 [Train: 0.971] [Test: 0.735] [Valid: 0.754]
11:45:29.943979
#294 [Train: 0.118] [Test: 1.649] [Valid: 1.370]
#294 [Train: 0.971] [Test: 0.739] [Valid: 0.754]
11:46:19.380415
#295 [Train: 0.118] [Test: 1.642] [Valid: 1.372]
#295 [Train: 0.970] [Test: 0.736] [Valid: 0.754]
11:47:08.789133
#296 [Train: 0.117] [Test: 1.646] [Valid: 1.383]
#296 [Train: 0.971] [Te

In [20]:
# label.shape

In [21]:
# output = model(torch.cuda.FloatTensor(batch))

In [22]:
# output.shape

In [23]:
# torch.tensor(batch).unsqueeze_(1).shape

In [24]:
# input_tensor = torch.tensor(batch).to(device).float()
# input_tensor = input_tensor.transpose(1, 2)
# model(input_tensor).shape

In [25]:
# for batch, label_list in batch_generator:
#     print(batch.shape[0])
#     plt.figure(figsize=(5, 1))
#     plt.plot(label_list)
#     plt.show()
    
#     plt.figure(figsize=(6, 18))
# #     plt.imshow(batch[0], aspect='auto')
#     plt.imshow(batch[0])
#     plt.show()
    
#     break